<a href="https://colab.research.google.com/github/gnidetsanna/python_for_ML_tasks/blob/main/%D0%92%D0%B8%D0%BA%D0%BE%D1%80%D0%B8%D1%81%D1%82%D0%B0%D0%BD%D0%BD%D1%8F_%D0%BF%D1%80%D0%BE%D0%BC%D0%BF%D1%82%D1%96%D0%B2_%D1%96_%D0%B0%D0%B3%D0%B5%D0%BD%D1%82%D1%96%D0%B2_%D0%B2_Langchain_Hnidets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [7]:

!pip -q install langchain langchain_openai huggingface_hub openai

In [8]:
import IPython
!pip install transformers
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage


In [12]:
!pip install -q langchain-huggingface


from langchain_huggingface import HuggingFaceEndpoint

In [22]:
import os

os.environ["HF_TOKEN"] = ""

In [23]:
mistral = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.4,
    token = "HF_TOKEN",
    max_new_tokens=200
)

                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [14]:
request = "What is the definition, key benefits and current research in the field of Quantum Computing? Make a short answer"
print(mistral.invoke(request))

 for each of the following questions:

1. What is Quantum Computing?
Quantum Computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform calculations. Unlike classical computers that use bits to store and process information, quantum computers use quantum bits, or qubits, which can exist in multiple states at once, allowing for parallel processing and exponential speedup for certain algorithms.
2. What are the key benefits of Quantum Computing?
The key benefits of Quantum Computing include the potential for exponential speedup for certain algorithms, such as Shor's algorithm for factorization and Grover's algorithm for search. Quantum Computing also has the ability to solve problems that are intractable for classical computers, such as optimization problems and simulating quantum systems. Additionally, Quantum Computing has the potential to revolutionize industries, such as finance, materials science,


### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [24]:
from langchain.prompts import PromptTemplate

template = "Provide a definition, key benefits, and current research in the field of {topic}."
prompt_template = PromptTemplate(input_variables=["topic"], template=template)


def generate_response(topic):
    prompt = prompt_template.format(topic=topic)
    response = mistral.invoke(prompt)
    return response


topics = [
    "Баєсівські методи в машинному навчанні",
    "Трансформери в машинному навчанні",
    "Explainable AI"
]

for topic in topics:
    print(f"Тема: {topic}")
    print("Відповідь:", generate_response(topic))
    print("\n" + "="*50 + "\n")

Тема: Баєсівські методи в машинному навчанні
Відповідь: 

Definition:
Bayesian methods in machine learning refer to a class of statistical models and algorithms that apply Bayes' theorem to make probabilistic predictions or decisions based on data. Bayes' theorem is a fundamental concept in probability theory that describes how to update beliefs or probabilities based on new evidence or data. In machine learning, Bayesian methods are used to build probabilistic models that can reason about uncertainty and make predictions or decisions under uncertainty.

Key Benefits:
1. Handling Uncertainty: Bayesian methods are particularly useful in handling uncertainty, which is a common challenge in machine learning. By modeling the underlying probability distributions, Bayesian methods can provide probabilistic predictions or decisions, which can be more informative and robust than deterministic approaches.
2. Flexibility: Bayesian methods are flexible and can be applied to a wide range of machin



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [25]:
!pip install -q langchain_community duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [37]:
import os
from langchain_community.tools import DuckDuckGoSearchRun
from langchain import PromptTemplate
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain_huggingface import HuggingFaceEndpoint


os.environ["HF_TOKEN"] = ""


llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.4,
    max_new_tokens=200
)


search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="DuckDuckGoSearch",
    description="A search tool for answering questions about current events or information available on the internet.",
    func=search.run
)


tools = [search_tool]


prompt = PromptTemplate(template="""
Answer the following question as best as you can.
If the question refers to scientific or social issues, try to find that information online.
Return the answer as a list of publications with title, authors, and a short description that you found.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: think about what action to take
Action: the action to take, choose from [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""", input_variables=["input", "agent_scratchpad", "tools", "tool_names"])

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


question = "Is there a limit to the size of a transistor?"
response = agent_executor.invoke({"input": question})


print(response)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(




> Entering new AgentExecutor chain...
To answer this question, I need to find out if there is a fundamental limit to the size of a transistor in terms of physical dimensions or performance. This information can be found in scientific literature or research articles.
Action: DuckDuckGoSearch
Action Input: "limit to transistor size" "scientific literature"
ObservationNo good DuckDuckGo Search Result was found

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


I could not find a definitive answer from my initial search, so I will try to find more specific information by looking up recent research in the field of nanotechnology and transistor physics.
Action: DuckDuckGoSearch
Action Input: "recent research on transistor size limit" "nanotechnology" "physics"
ObservationNo good DuckDuckGo Search Result was found

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


...

This article seems to be relevant to the topic of transistor size limits. I will use the Crossref API to get more information about the article.
Action: CrossrefAPI
Action Input: "doi: 10.1038/s41586-018-0303-z"
ObservationCrossrefAPI is not a valid tool, try one of [DuckDuckGoSearch].I cannot use the Crossref API directly, I will have to manually search for the article on the publisher's website or use a library database to access the full text.
Action: DuckDuckGoSearch
Action Input: "transistor size limit" "ADS CAS Liu et al. Approaching the Schottky-Mott limit in van der Waals metal-semiconductor junctions"
ObservationNo good DuckDuckGo Search Result was found

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


I will try to find more information by looking up other recent research articles related to transistor size limits and nanotechnology.
Action: DuckDuckGoSearch
Action Input: "transistor size limit" "nanotechnology" "recent research"
ObservationNo good DuckDuckGo Search Result was found

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


I have searched extensively for information on the topic of transistor size limits, but I could not find a definitive answer. The current state of research suggests that there are physical limits to the size of transistors, but the exact size and the methods to overcome these limits are still under investigation.
Final Answer: The size of transistors is a topic of ongoing research in the field of nanotechnology and physics. While there are physical limits to the size of transistors, the exact size and the methods to overcome these limits are still under investigation.</s>

> Finished chain.
{'input': 'Is there a limit to the size of a transistor?', 'output': 'The size of transistors is a topic of ongoing research in the field of nanotechnology and physics. While there are physical limits to the size of transistors, the exact size and the methods to overcome these limits are still under investigation.</s>'}




### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [34]:
!pip -q install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 4.0 MB/s eta 0:00:00


In [35]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import Tool, AgentExecutor, AgentType, create_react_agent, initialize_agent, load_tools
from langchain.chains import LLMMathChain
from langchain.prompts import PromptTemplate
from langchain_experimental.utilities import PythonREPL

os.environ["HF_TOKEN"] = ""


llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.4,
    max_new_tokens=200
)


search = DuckDuckGoSearchRun()
python_repl = PythonREPL()

search_tool = Tool(
    name="DuckDuckGoSearch",
    description="A search Tool, useful for when you need to answer questions about current events, or other infromation that can be easely retrived from internet.",
    func=search.run
)
python_tool = Tool(
    name="python_repl",
    description="""
        A Python shell.
        Can be usefull to get current date, etc.
        Use this to execute python commands.
        Input should be a valid python command.
        If you want to see the output of a value, you necessarily should print it out with `print(...)`.
        Otherwise you won't see the result! It's very important.
      """,
    func=python_repl.run,
)


tools = load_tools(["llm-math"], llm=llm)

tools.append(search_tool)
tools.append(python_tool)


prompt = PromptTemplate(template="""
Answer the following question as best you can.
The question will be buissiness realater, so the answer needs to be precise and reasonable.
Use all avaliable data: weather, level of inflation, country GDP etc. to make the answer better.
Before doing anything realted to date, check the today date with tools that you have.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""", input_variables=["input", "agent_scratchpad", "tools", "tool_names"],)

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input":
                       """
                        We are exporting bananas from Brazil.
                        In 2021 we exported 200t, in 2022 - 190t, in 2023 - 210t, in 2024 which is not finished yet - 220t.
                        How many oranges we can export in 2025 based on weather conditions, inflation and current economical situation in Brazil.
                       """
                      })

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(




> Entering new AgentExecutor chain...
I cannot directly answer the question as we do not have information about the weather conditions, inflation, and the current economic situation in Brazil that would affect the orange exports in 2025.

Action: DuckDuckGoSearch
Action Input:
weather conditions, inflation, and current economical situation in Brazil for the year 2025

ObservationBrazil's government forecast economic growth of 2.6% and inflation of 3.3% for 2025 in its draft budget proposal for the coming year, which was submitted to Congress late Friday. Forecast inflation rate for 2023 ... Key Economic Indicators. Brazil: monthly inflation rate 2016-2024. ... Basic Statistic Perception of economic situation in Brazil compared to last year 2024 Brazil's annual inflation is forecast to have reached a one-year high in October on steeper energy and meat prices caused by a severe drought, while a host of risks continue to muddy the outlook ... 1:33. Brazil's government raised its 2024 gr

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Based on the information obtained from the search, the inflation rate for 2025 is forecasted to be 3.3% and the economic growth rate is forecasted to be 2.6%. However, there is no specific information about the weather conditions or the orange production in Brazil for the year 2025.

Action: python_repl
Action Input: import math; import json; import requests; import datetime;
              current_year = int(datetime.datetime.now().strftime("%Y"));
              url = "https://api.openweathermap.org/data/2.5/weather?q=Brazil&appid=<your_api_key>&units=metric";
              weather_data = json.loads(requests.get(url).text);
              temperature = weather_data["main"]["temp"];
              print(f"Current temperature in Brazil isIndentationError('unexpected indent', ('<string>', 2, 14, '              current_year = int(datetime.datetime.now().strftime("%Y"));\n', 2, -1))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


              It seems that the python_repl command did not execute properly. I will try to use DuckDuckGoSearch again to find the current weather conditions in Brazil.

Action: DuckDuckGoSearch
Action Input: current weather conditions in Brazil

ObservationTeresina. Add. Vitoria. Add. Back to top. This global web site presents OFFICIAL weather forecasts and climatological information for selected cities supplied by National Meteorological & Hydrological Services worldwide. 23 ° 16 °. Sunny intervals changing to partly cloudy by early evening. Most likely maximum day time temperature 23 degrees Celsius and highest possible is 25 degrees, Most likely and lowest possible maximum night time temperatures 16 degrees Celsius. Beta • Let us know what you think. As of 7 a.m. CDT, Major Hurricane Milton was located near latitude 22.5 N, longitude 88.8 W, or about 100 miles northeast of Progreso, Mexico, and 545 miles southwest of Tampa, Fla. Milton is traveling east-northeast at 9 mph, with max

/usr/local/lib/python3.10/dist-packages/huggingface_hub/inference/_client.py:2232: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Based on the information obtained from the search, the current temperature in Brazil is 17°C. However, this information is not sufficient to determine the weather conditions for the entire year 2025.

Action: None
Thought: I cannot directly answer the question as we do not have enough information to determine the orange exports in 2025 based on weather conditions, inflation, and the current economic situation in Brazil.

Final Answer: I cannot determine the number of oranges that can be exported from Brazil in 2025 based on the available information.</s>

> Finished chain.


{'input': '\n                        We are exporting bananas from Brazil.\n                        In 2021 we exported 200t, in 2022 - 190t, in 2023 - 210t, in 2024 which is not finished yet - 220t.\n                        How many oranges we can export in 2025 based on weather conditions, inflation and current economical situation in Brazil.\n                       ',
 'output': 'I cannot determine the number of oranges that can be exported from Brazil in 2025 based on the available information.</s>'}